In [ ]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [ ]:

# Read the text file
data_path = '/Users/steffin/Documents/data-scribbles/projects/p02/data_next_word_predictor.txt'
with open(data_path, 'r', encoding='utf-8') as file:
    word_data = file.read()
tokenizer = Tokenizer()
tokenizer.fit_on_texts([word_data])
total_words = len(tokenizer.word_index) + 1
input_sequences = []
for line in word_data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs=100, verbose=1)
model.save("/Users/steffin/Documents/data-scribbles/projects/p02/100epoch_next_word_predictor_model.h5")